## Points to remember

Write down the points to be remembered in this cell while coding, so it might be helpful to others working on same code - different functions

1) Every current node (object) will be updated as visited after it goes through inference rules function <br>
2) send 3 arguments to init()<br>
3) import this file to the main file<br>
4) declaring only one set of variables for: agent_matrix,helper_matrix, agent_gridworld, full_gridworld in the main file. This file is only to implement functionalities of inference engine<br>

In [ ]:
import numpy as np
import sympy as sym

In [5]:
class partial_sensing:

#         self.Nx = Nx   the number of neighbors cell x has.
#         self.visited = visited  Whether or not cell x has been visited.
#         self.blocked = blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed.
#         self.Cx = Cx   the number of neighbors of x that are sensed to be blocked.
#         self.Bx = Bx   the number of neighbors of x that have been confirmed to be blocked..
#         self.Ex = Ex   the number of neighbors of x that have been confirmed to be empty.
#         self.Hx = Hx  the number of neighbors of x that are still hidden or unconfirmed either way.

    def __init__(self, row, col, maze_size):
        self.row = row
        self.column = col
        self.neighbor_list = [] #need to find a node's neighbor indices and append to this list
        self.Nx = 0
        self.Cx = 0
        self.Bx = 0
        self.Ex = 0
        self.Hx = 0
        self.visited = False
        self.blocked = 0
        self.free_neighbors = []
        self.blocked_neighbors = []

    def display_node(self):
        print("Node: ", self.row, "," , self.column)
        print("\n the number of neighbors cell x has: ", self.Nx)
        print("visited  Whether or not cell x has been visited: ", self.visited)
        print("blocked  Whether or not cell x has been confirmed as empty or blocked, or is currently unconfirmed: ", self.blocked)
        print("the number of neighbors of x that are sensed to be blocked: ", self.Cx)
        print("the number of neighbors of x that have been confirmed to be blocked: ", self.Bx)
        print("the number of neighbors of x that have been confirmed to be empty: ", self.Ex)
        print("the number of neighbors of x that are still hidden or unconfirmed either way: ", self.Hx)
        print("The undiscovered neighbors are: ", self.neighbor_list)
        print("The confirmed neighbors that are free: ", self.free_neighbors)
        print("The confirmed neighbors that are blocked: ", self.blocked_neighbors)

    def inference_engine(self, visited_list, infered_list, knowledge_base, expression_list, agent_gridworld):
        visited_plus_infered_list = []
        list_of_neighbor_status = []
        expr = ""
        visited_plus_infered_list.extend(visited_list)
        visited_plus_infered_list.extend(infered_list)

        # if current node is blocked substitute current node's blocked status into expressions and return
        # we donot compute the Cx value for the blocked node
        if self.blocked == 1:
            for expression in expression_list:
                node_to_substitute = sym.Symbol('node(' + str(self.row) + str(self.column) + ')')
                index = expression_list.index(expression)
                expression_list[index] = expression.subs(node_to_substitute, self.blocked)
            return knowledge_base, infered_list, expression_list, agent_gridworld

        # initializing an expression
        # if the current node is free, we compute an expression as "self.Cx = neighbor_1.blocked + neighbor_2.blocked.."
        # i.e., The equation becomes => neighbor_1.blocked + neighbor_2.blocked .. - self.Cx = 0
        if self.visited == False:
            a = sym.Symbol('node(' + str(self.row) + str(self.column) + ')')
            for neighbor in self.neighbor_list:
                neighbor_row = neighbor[0]
                neighbor_col = neighbor[1]
                neighbor_node_object = knowledge_base[neighbor_row][neighbor_col]
                if not expr:
                    expr = sym.Symbol('node(' + str(neighbor_node_object.row) + str(neighbor_node_object.column) + ')')
                else:
                    expr += sym.Symbol('node(' + str(neighbor_node_object.row) + str(neighbor_node_object.column) + ')')
            expr -= a
            expr = expr.subs(a, self.Cx)
            # expression_list has the list of expressions to be computed at any given point in time
            expression_list.append(expr)
        
        # substitute blocked values for previously visited nodes
        for expression_index in range(len(expression_list)):
            expression = expression_list[expression_index]
            #updating current node's value in the expression
            for neighbor in visited_plus_infered_list:
                neighbor_row = neighbor[0]
                neighbor_col = neighbor[1]
                neighbor_node_object = knowledge_base[neighbor_row][neighbor_col]
                node_to_substitute = sym.Symbol('node(' + str(neighbor_node_object.row) + str(neighbor_node_object.column) + ')')
                substituted_expression = expression.subs(node_to_substitute, neighbor_node_object.blocked)
                substituted_expression = substituted_expression.subs(node_to_substitute, self.blocked)
                expression_list[expression_index] = substituted_expression
                expression = substituted_expression

        # solving the expression
        for expression in expression_list:
            # condition 1 if the expression evaluates to zero the single node in the expression is free
            if sym.solve(expression)[0] == 0:
                # fetch the node's row and column values adn retrieve it's respective object from knowledge base
                print("Infered the node is: ", str(expression))
                infered_row = int(str(expression)[5:7][0])
                infered_col = int(str(expression)[5:7][1])
                infered_object = knowledge_base[infered_row][infered_col]
                
                # if the node isn't attached to visited or infered list we append it to solve the equations for next iteration
                if (infered_row, infered_col) not in visited_plus_infered_list:
                    infered_list.append((infered_row, infered_col))
                expression_list.remove(expression)
                knowledge_base[infered_row][infered_col].blocked = 0   # updating the knowledge base 
            # condition 2 if the expression evaluates to one the single node in the expression is blocked
            elif sym.solve(expression)[0] == 1:
                # fetch the node's row and column values adn retrieve it's respective object from knowledge base
                print("Infered the node is: ", str(expression))
                infered_row = int(str(expression)[5:7][0])
                infered_col = int(str(expression)[5:7][1])
                infered_object = knowledge_base[infered_row][infered_col]

                # if the node isn't attached to visited or infered list we append it to solve the equations for next iteration
                if (infered_row, infered_col) not in visited_plus_infered_list:
                    infered_list.append((infered_row, infered_col))
                expression_list.remove(expression)
                knowledge_base[infered_row][infered_col].blocked = 0  # updating the knowledge base 
            # condition 3 if node1 = node 2 then both the nodes are free eg:- -node(21) = -node(23)
            elif sym.solve(list(sym.solve(expression)[0].values())[0])[0] == 0:
                
                # node(23)
                infered_row1 = int(str(list(sym.solve(expression)[0].values())[0]).replace("-","")[5:7][0])
                infered_col1 = int(str(list(sym.solve(expression)[0].values())[0]).replace("-","")[5:7][1])
                
                infered_row2 = int(str(list(sym.solve(expression)[0].keys())[0]).replace("-","")[5:7][0])
                infered_col2 = int(str(list(sym.solve(expression)[0].keys())[0]).replace("-","")[5:7][1])

                knowledge_base[infered_row1][infered_col1].blocked = 0   # updating the knowledge base 
                
                # if the node isn't attached to visited or infered list we append it to solve the equations for next iteration
                if (infered_row1, infered_col1) not in visited_plus_infered_list:
                    infered_list.append((infered_row1, infered_col1))
                    
                knowledge_base[infered_row2][infered_col2].blocked = 0   # updating the knowledge base 
                
                # if the node isn't attached to visited or infered list we append it to solve the equations for next iteration
                if (infered_row2, infered_col2) not in visited_plus_infered_list:
                    infered_list.append((infered_row2, infered_col2))
                    
                expression_list.remove(expression)

                print('infered node is: ', list(sym.solve(expression)[0].values())[0])
                print('infered node is: ', list(sym.solve(expression)[0].values())[0])

        return knowledge_base, infered_list, expression_list, agent_gridworld